## Steps that I used to perform model training, evaluating, inferencing and exporting the model using Tao toolkit on AWS EC2 cloud setup



- It is expected that the TAO toolkit,the related dependecies and pre-trained models are already installed on the setup, instructions are given in below document:

#### TAO Quick start guide
https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_quick_start_guide.html 


#### DetectNet_v2 model

Reference: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html 


## The below steps were performed in a similar manner for all the three approaches.

### Step 1 TAO dependencies

It is expected that the TAO toolkit and the related dependecies are already installed on the setup, instructions are given in below document:
https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_quick_start_guide.html 

### Step 2 Pre-processing data-set

Create below directory structure to store data and configuration files

USER_EXPERIMENT_DIR=/workspace/tao-experiments/detectnet_v2_ppe

DATA_DOWNLOAD_DIR=/workspace/tao-experiments/detectnet_v2_ppe/data

SPECS_DIR=/workspace/tao-experiments/detectnet_v2_ppe/specs

TRAINING_DIR=/workspace/tao-experiments/detectnet_v2_ppe/training




### Step 3 Data directory

DATA_DOWNLOAD_DIR, this directory will contain below subfolder:

1) train:

- This folder contains images and labels directory which will be used for model training

2) test:

- This folder contains images and labels directory which will be used for model evaluation on test dataset

3) val:

- This folder contains images and labels directory which will be used for model evaluation on validation dataset

4) tfrecords:

- This folder contains tfrecord files for train, test and validation dataset



SPECS_DIR=/workspace/tao-experiments/detectnet_v2_ppe/specs

SPECS_DIR, this directory will contain below files:

- training.cfg

- inference.cfg

- train_convert.cfg

- test_convert.cfg

- val_convert.cfg



### Step 3 Tfrecords generate

Create train_convert.cfg, test_convert.cfg and val_convert.cfg files for TAO to identify folders of dataset that needs to be converted to tfrecords

Configuration file for dataset convert: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html#configuration-file-for-dataset-converter

Sample command to generate training data-set tfrecord:
>> tao detectnet_v2 dataset_convert -d /workspace/tao-experiments/detectnet_v2_ppe/specs/train_convert.cfg \ -o /workspace/tao-experiments/detectnet_v2_ppe/data/tfrecords/train/ -v


Similarly for validation dataset:
>> tao detectnet_v2 dataset_convert -d /workspace/tao-experiments/detectnet_v2_ppe/specs/val_convert.cfg \ -o /workspace/tao-experiments/detectnet_v2_ppe/data/tfrecords/val/ -v

Similarly for Test dataset:
>> tao detectnet_v2 dataset_convert -d /workspace/tao-experiments/detectnet_v2_ppe/specs/test_convert.cfg \ -o /workspace/tao-experiments/detectnet_v2_ppe/data/tfrecords/test/ -v


### Step 4 Create training configuration file

Training configuration document: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html#model-config 

training.cfg file can be found under specs directory

SPECS_DIR=/workspace/tao-experiments/detectnet_v2_ppe/specs




### Step 5 Model training

Sample command used for running model training:
>> tao detectnet_v2 train -e 
/workspace/tao-experiments/detectnet_v2_ppe/specs/training.cfg -r /workspace/tao-experiments/detectnet_v2_ppe/training_run/ -k tlt_encode --gpus 1 --gpu_index 2 -v

In [3]:
# Output of above model training after 100/100 epochs:
# Validation cost: 0.000713
# Mean average_precision (in %): 63.9297
 
# class name      average precision (in %)
# ------------  --------------------------
# helmet                           53.5013
# person                           70.117
# vest                             68.1708

At the end of training, tao will generate model.tlt file that can be used for model evaluation.

### Step 6 Model Evaluation

Model evaluation config: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html#configuring-the-evaluator


Sample command to run model evaluation:
>> tao detectnet_v2 evaluate -e /workspace/tao-experiments/detectnet_v2_ppe/specs/training.cfg -m /workspace/tao-experiments/detectnet_v2_ppe/ppe_training/weights/model.tlt -k tlt_encode


In [4]:
# Evaluation on Validation data

# Validation cost: 0.000877
# Mean average_precision (in %): 62.9232

# class name      average precision (in %)
# ------------  --------------------------
# helmet                           54.56
# person                           67.4151
# vest                             66.7945

# =====================================================

# Evaluation on Test data:
# Validation cost: 0.001008
# Mean average_precision (in %): 60.7581

# class name      average precision (in %)
# ------------  --------------------------
# helmet                           50.8559
# person                           68.3887
# vest                             63.0295


### Step 7 Model Inference

Document: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html#specification-file-for-inference 


Sample command to generate inference files using trained model:

For validation dataste
>> tao detectnet_v2 inference -e /workspace/tao-experiments/detectnet_v2_ppe/specs/inference.cfg -i /workspace/tao-experiments/detectnet_v2_ppe/data/val/images/ -o /workspace/tao-experiments/detectnet_v2_ppe/data/val/labels/ -k tlt_encode

For test dataset:
>> tao detectnet_v2 inference -e /workspace/tao-experiments/detectnet_v2_ppe/specs/inference.cfg -i /workspace/tao-experiments/detectnet_v2_ppe/data/test/images/ -o /workspace/tao-experiments/detectnet_v2_ppe/data/test/labels/ -k tlt_encode





### Step 8 Generating the calibration_tensor file in Int8 mode

>> tao detectnet_v2 calibration_tensorfile -e /workspace/tao-experiments/detectnet_v2_ppe/specs/training.cfg -m 10 -o /workspace/tao-experiments/detectnet_v2_ppe/ppe_training/weights/Int8/calibration.tensor


### Step 9 Exporting the model

Sample command:
>> tao detectnet_v2 export -m /workspace/tao-expeiments/ppe_classification/ppe_training/weights/model.tlt 
-k tlt_encode -e /workspace/tao-experiments/detectnet_v2_ppe/specs/training.cfg -o /workspace/tao-experiments/detectnet_v2_ppe/ppe_training/weights/export/Int8/resnet18_detector.etlt --cal_data_file /workspace/tao-experiments/detectnet_v2_ppe/ppe_training/weights/export/Int8/calibration.tensor --data_type int8 --batches 10 --batch_size 4 --max_batch_size 4 --engine_file /workspace/tao-experiments/detectnet_v2_ppe/ppe_training/weights/export/Int8/resnet18_detector.trt.int8 --cal_cache_file /workspace/tao-experiments/detectnet_v2_ppe/ppe_training/weights/export/Int8/calibration.bin --gen_ds_config

In [6]:
# After export below files will be created:
# (ppe_detector) jaydeep@ip-172-31-29-173:/workspace/tao-experiments/detectnet_v2_ppe/ppe_training/weights/export/Int8$ ls -lrt
# total 167452
# -rw-rw-rw- 1 jaydeep jaydeep 112138546 Jul  7 10:21 calibration.tensor
# -rw-rw-rw- 1 jaydeep jaydeep  46311310 Jul  7 10:42 resnet18_detector.etlt
# -rw-rw-rw- 1 jaydeep jaydeep      4864 Jul  7 10:43 calibration.bin
# -rw-rw-rw- 1 jaydeep jaydeep  12997755 Jul  7 10:44 resnet18_detector.trt.int8
# -rw-rw-rw- 1 jaydeep jaydeep       289 Jul  7 11:01 nvinfer_config.txt
# -rw-rw-rw- 1 jaydeep jaydeep        19 Jul  7 11:01 labels.txt


### Step 10 Model deployment using DeepStream on Jetson device

Reference document for deploying in Deepstream: https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/detectnet_v2.html#deploying-to-deepstream

After required model files are created, these files will be downloaded in Jetson Xavier NX and real-time inference can be run using DeepStream SDK installed on the Jetson device